# **Colab환경에서 실행해야합니다**

In [1]:
%%capture
# Normally using pip install unsloth is enough

# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any! We auto support RoPE Scaling internally!
dtype = (
    None  # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
)
load_in_4bit = True  # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-2-9b",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Gemma2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized

    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.1.8 patched 42 layers with 42 QKV layers, 42 O layers and 42 MLP layers.


In [4]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for input, output in zip(inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = f'{input}\n{output}' + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset('json', data_files={'train':'bccard.json'})
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/31250 [00:00<?, ? examples/s]

In [5]:
dataset['train']['text'][0]

"비씨카드는 어떤 회사인가요?\n비씨카드(BC카드)는 1982년에 설립된 대한민국의 대표적인 카드사로, 주로 결제 프로세싱 서비스를 제공하는 회사입니다. 설립 초기에는 '은행신용카드협회'라는 이름으로 시작했으며, BC는 'Bank Credit'의 약자입니다. 이 협회는 조흥은행, 상업은행, 서울은행, 제일은행, 한일은행 등 주요 은행들이 공동으로 설립하여, 카드 결제 시스템을 효율적으로 관리하고, 국내 카드 결제 인프라를 구축하는 역할을 했습니다.\n\n비씨카드는 이후 독립적인 신용카드 회사로 성장하며, 카드 발급, 결제 처리, 가맹점 관리 등의 다양한 금융 서비스를 제공하게 되었습니다. 현재 비씨카드는 KT가 최대 주주로, KT의 빅데이터 및 AI 기술을 활용해 디지털 금융 서비스를 강화하고 있습니다. 비씨카드는 결제 프로세싱 시장에서 오랜 경험과 노하우를 바탕으로, 국내외에서 다양한 금융 서비스를 제공하며 카드사 중에서도 중요한 위치를 차지하고 있습니다.\n\n비씨카드는 단순히 카드 발급과 결제에 그치지 않고, 디지털 결제 플랫폼 '페이북'을 운영하며, 중금리 대출, 마이데이터 사업 등으로 서비스 범위를 확장하고 있습니다. 이러한 디지털 혁신은 비씨카드가 빅테크 기업들과의 경쟁에서도 경쟁력을 유지하기 위한 핵심 전략입니다. \n\n이처럼 비씨카드는 전통적인 카드사로서의 역할을 유지하면서도, 디지털 금융의 선두주자로 자리매김하기 위해 지속적으로 변화를 추구하고 있는 기업입니다.<eos>"

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/31250 [00:00<?, ? examples/s]

In [7]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 31,250 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 54,018,048


Step,Training Loss
1,1.082700
2,1.300100
3,1.403700
4,1.368500
5,1.494200
6,1.341900
7,1.325700
8,1.059800
9,1.417000
10,1.378700


In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer('주식 투자를 위해선 무엇을 준비해야 하나요?', return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, no_repeat_ngram_size=2)

In [19]:
result = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n')

print('질문 :', result[0])
print()
print('답변 :', result[1])

질문 : 주식 투자를 위해선 무엇을 준비해야 하나요?

답변 : 주가 투자가 성공하기 위해서는 먼저 투기성이 아닌 장기적인 투자인지를 확인해야 합니다. 또한, 투수금을 조정하고, 주식 시장에 대한 이해를 넓히는 것이 중요합니다.
